<a href="https://colab.research.google.com/github/LennRBriones/neural-networks-practices/blob/main/solving_regretion_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prediction house prices

In [2]:
import pandas as pd
import numpy as pd
from keras.datasets import boston_housing
from keras import layers, models, optimizers

In [3]:
(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [4]:
train_data.shape

(404, 13)

In [5]:
train_labels.shape

(404,)

In [6]:
train_labels[0]

15.2